In [36]:
import base64
import json
import random

import pandas as pd
import requests
import tensorflow as tf

In [37]:
df = pd.read_csv("data/loan_data.csv")
y = df.pop("loan_status")

columns = df.columns.values
rand = random.randint(0, len(df))

features = df.values[rand]
label = y[rand]

inputs = {key: value for key, value in zip(columns, features)}
inputs

{'person_age': 24.0,
 'person_gender': 'male',
 'person_education': 'Bachelor',
 'person_income': 91159.0,
 'person_emp_exp': 3,
 'person_home_ownership': 'MORTGAGE',
 'loan_amnt': 15600.0,
 'loan_intent': 'VENTURE',
 'loan_int_rate': 11.01,
 'loan_percent_income': 0.17,
 'cb_person_cred_hist_length': 4.0,
 'credit_score': 654,
 'previous_loan_defaults_on_file': 'Yes'}

In [38]:
def string_feature(value):
    return tf.train.Feature(
        bytes_list=tf.train.BytesList(value=[bytes(value, "utf-8")]),
    )


def float_feature(value):
    return tf.train.Feature(
        float_list=tf.train.FloatList(value=[value]),
    )


def int_feature(value):
    return tf.train.Feature(
        int64_list=tf.train.Int64List(value=[value]),
    )

In [39]:
def prepare_json(inputs: dict):
    feature_spec = dict()
    
    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = float_feature(values)
        elif isinstance(values, int):
            feature_spec[keys] = int_feature(values)
        elif isinstance(values, str):
            feature_spec[keys] = string_feature(values)
            
    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()
        
    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]
    
    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

In [40]:
def make_predictions(inputs):
    json_data = prepare_json(inputs)

    endpoint = (
        "https://loan-mlops-dicoding-production.up.railway.app/v1/models/loan-approval-model:predict"
    )
    response = requests.post(endpoint, data=json_data)
    prediction = response.json()["predictions"][0][0]

    if prediction < 0.7:
        return "persetujuan pinjaman tidak diterima"
    else:
        return "persetujuan pinjaman diterima"

In [41]:
make_predictions(inputs)

'persetujuan pinjaman tidak diterima'

In [42]:

label

0